# Анализ базы данных книжного сервиса

__ЦЕЛЬ ИССЛЕДОВАНИЯ__

Провести анализ базы данных для формулирования ценностного предложения нового продукта.

__ЗАДАЧИ ИССЛЕДОВАНИЯ__

С помощью SQL-запросов к базе данных:
- Посчитать сколько книг вышло после 1 января 2000 года.
- Для каждой книги посчитать количество обзоров и среднюю оценку.
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц.
- Определить автора с самой высокой средней оценкой книг для книг с 50 и более оценками.
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

__ОПИСАНИЕ ДАННЫХ__

Таблица `books`. Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`. Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`. Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

 Таблица `ratings`. Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`. Содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

<div style="border:solid darkblue 2px; padding: 20px">    
СХЕМА ДАННЫХ
 <img src="//pictures.s3.yandex.net/resources/scheme_1589269096.png" />
</div>

__ПЛАН РАБОТЫ__
1. Загрузить и настроить необходимые библиотеки.
2. Подключиться к базе данных.
3. Прочитать и проверить необходимые таблицы.
4. Последовательно выполнить все задачи проекта.
5. Составить выводы.

## Загрузка и настройка библиотек

Для данной работы нам необходимо совсем немного библиотек.

In [1]:
# импорт библиотек
import pandas as pd
import sqlalchemy as sa

print('Библиотеки успешно загружены!')

Библиотеки успешно загружены!


## Подключение к базе данных

Зададим нужные параметры для подключения к базе данных.

In [2]:
# установка параметров конфигурации для доступа к базе данных (данные конфиденциальны)
db_config = {
            'user': , # имя пользователя
            'pwd': , # пароль
            'host': ,
            'port': , # порт подключения
            'db': '' # название базы данных
            }
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохранение коннектора
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

Для удобства дальнейшей работы зададим простую функцию выдачи результата запроса.

In [3]:
# создание функции
def def_get_sql(query):
    # описание функции
    """
    Эта функция открывает соединение, получает данные из SQL-запроса, закрывает соединение.
    
    Принимает на вход:
    - query - SQL-запрос
    
    """
    # получение данных
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Чтение и проверка исходных таблиц

Посмотрим на все исходные таблицы: с помощью SQL-запроса выведем первые 5 строк каждого датасета.

In [4]:
# создание SQL-запроса
query = '''
SELECT *
FROM books
LIMIT 5

'''

# вывод заголовка
print("\033[1;31mТаблица `books` - примеры строк:\033[0m")

# вывод результата запроса
display(def_get_sql(query))

Таблица `books` - примеры строк:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
# создание SQL-запроса
query = '''
SELECT COUNT(DISTINCT book_id) AS count_books,
       COUNT(DISTINCT author_id) AS count_authors,
       COUNT(DISTINCT publisher_id) AS count_publishers,
       MAX(num_pages) AS max_pages,
       MIN(num_pages) AS min_pages,
       MIN(publication_date) AS min_date,
       MAX(publication_date) AS max_date
FROM books

'''

# получение результата запроса
result = def_get_sql(query)

# вывод заголовка
print("\033[1;31mТаблица `books` - обзор данных:\033[0m")

# вывод результата запроса
print('- Количество книг:', result['count_books'].values[0])
print('- Количество авторов:', result['count_authors'].values[0])
print('- Количество издательств:', result['count_publishers'].values[0])
print('- Минимальное количество страниц в книге:', result['min_pages'].values[0])
print('- Максимальное количество страниц в книге:', result['max_pages'].values[0])
print('- Самая ранняя дата публикации:', result['min_date'].values[0])
print('- Самая поздняя дата публикации:', result['max_date'].values[0])

Таблица `books` - обзор данных:
- Количество книг: 1000
- Количество авторов: 636
- Количество издательств: 340
- Минимальное количество страниц в книге: 14
- Максимальное количество страниц в книге: 2690
- Самая ранняя дата публикации: 1952-12-01
- Самая поздняя дата публикации: 2020-03-31


In [6]:
# создание SQL-запроса
query = '''
SELECT *
FROM authors
LIMIT 5

'''

# вывод заголовка
print("\033[1;31mТаблица `authors` - примеры строк:\033[0m")

# вывод результата запроса
display(def_get_sql(query))

Таблица `authors` - примеры строк:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
# создание SQL-запроса
query = '''
SELECT COUNT(author_id) AS count_authors
FROM authors

'''

# получение результата запроса
result = def_get_sql(query)['count_authors'].values[0]


# вывод заголовка
print("\033[1;31mТаблица `authors` - обзор данных:\033[0m")

# вывод результата запроса
print(f"В таблице представлено {result} авторов.")

Таблица `authors` - обзор данных:
В таблице представлено 636 авторов.


In [8]:
# создание SQL-запроса
query = '''
SELECT *
FROM publishers
LIMIT 5

'''

# вывод заголовка
print("\033[1;31mТаблица `publishers` - примеры строк:\033[0m")

# вывод результата запроса
display(def_get_sql(query))

Таблица `publishers` - примеры строк:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
# создание SQL-запроса
query = '''
SELECT COUNT(publisher_id) AS count_publishers
FROM publishers

'''

# получение результата запроса
result = def_get_sql(query)['count_publishers'].values[0]


# вывод заголовка
print("\033[1;31mТаблица `publishers` - обзор данных:\033[0m")

# вывод результата запроса
print(f"В таблице представлено {result} издательств.")

Таблица `publishers` - обзор данных:
В таблице представлено 340 издательств.


In [10]:
# создание SQL-запроса
query = '''
SELECT *
FROM ratings
LIMIT 5

'''

# вывод заголовка
print("\033[1;31mТаблица `ratings` - примеры строк:\033[0m")

# вывод результата запроса
display(def_get_sql(query))

Таблица `ratings` - примеры строк:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
# создание SQL-запроса
query = '''
SELECT COUNT(DISTINCT book_id) AS count_books,
       COUNT(DISTINCT username) AS count_user,
       COUNT(rating_id) as count_rating
FROM ratings

'''

# получение результата запроса
result = def_get_sql(query)

# вывод заголовка
print("\033[1;31mТаблица `rating` - обзор данных:\033[0m")

# вывод результата запроса
print('В таблице `rating` представлена информация о',
      result['count_rating'].values[0], 'оценках для',
      result['count_books'].values[0], 'книг от',
      result['count_user'].values[0], 'пользователей.'
     )

Таблица `rating` - обзор данных:
В таблице `rating` представлена информация о 6456 оценках для 1000 книг от 160 пользователей.


In [12]:
# создание SQL-запроса
query = '''
SELECT *
FROM reviews
LIMIT 5

'''

# вывод заголовка
print("\033[1;31mТаблица `reviews` - примеры строк:\033[0m")

# вывод результата запроса
display(def_get_sql(query))

Таблица `reviews` - примеры строк:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [13]:
# создание SQL-запроса
query = '''
SELECT COUNT(DISTINCT book_id) AS count_books,
       COUNT(DISTINCT username) AS count_user,
       COUNT(review_id) as count_review
FROM reviews

'''

# получение результата запроса
result = def_get_sql(query)

# вывод заголовка
print("\033[1;31mТаблица `review` - обзор данных:\033[0m")

# вывод результата запроса
print('В таблице `review` представлена информация о',
      result['count_review'].values[0], 'обзорах для',
      result['count_books'].values[0], 'книг от',
      result['count_user'].values[0], 'пользователей.'
     )

Таблица `review` - обзор данных:
В таблице `review` представлена информация о 2793 обзорах для 994 книг от 160 пользователей.


<div style="border:solid green 2px; padding: 20px">

<b>ВЫВОДЫ по разделу</b><br><br>
С помощью SQL-запросов к удалённой базе данных были получены 5 таблиц:

    - `books`(содержит 1000 книг, 636 авторов и 340 издательств)
    - `authors` (содержит 636 авторов)
    - `publishers` (содержит 340 издательств)
    - `ratings` (содержит информацию о 6456 оценках для 1000 книг от 160 пользователей)
    - `review` (содержит 2793 обзора на 994 книги от 160 пользователей)
    
Для каждой таблицы были выведены первые 5 строк и общая количественная информация.
    
</div>

## Получение необходимой информации (SQL-запросы)

### Количество книг

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [14]:
# создание SQL-запроса
query = '''
SELECT
     COUNT (*) AS books_2000
FROM books
WHERE publication_date >= '2000-01-01'

'''

# получение результата запроса
result = def_get_sql(query)['books_2000'].values[0]

# вывод результата запроса
print(f"После 1 января 2000 года вышла {result} книга.")

После 1 января 2000 года вышла 821 книга.


Больше 80% книг вышло за последний год (представленный в данных).

### Количество обзоров и средняя оценка

Для каждой книги посчитаем количество обзоров и среднюю оценку.

Сначала посчитаем обзоры.

In [15]:
# создание SQL-запроса
query = '''
SELECT  b.book_id,
        b.title,
        COUNT (DISTINCT r.review_id) AS count_review,
        AVG(rt.rating) AS avg_rating
FROM  books b
LEFT JOIN ratings rt ON b.book_id=rt.book_id
LEFT JOIN reviews r ON b.book_id=r.book_id
GROUP BY b.book_id, b.title
ORDER BY count_review DESC

'''

# получение результата запроса
result = def_get_sql(query)

# вывод заголовка
print("\033[1;31mКОЛИЧЕСТВО ОБЗОРОВ ДЛЯ КАЖДОЙ КНИГИ\n(вывод первых 5 строк)\033[0m")

# вывод результата запроса
display(result.head())


# вывод заголовка
print("\033[1;31mСРЕДНИЙ РЕЙТИНГ ДЛЯ КАЖДОЙ КНИГИ\n(вывод первых 5 строк)\033[0m")

# вывод результата запроса
display(result.sort_values(by='avg_rating', ascending=False).head())

# вывод информации
max_rating = result['avg_rating'].max()
print(f"Максимальный рейтинг `{max_rating}`",
      f"у {len(result.query('avg_rating == @max_rating'))} книг.")


КОЛИЧЕСТВО ОБЗОРОВ ДЛЯ КАЖДОЙ КНИГИ
(вывод первых 5 строк)


,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081


СРЕДНИЙ РЕЙТИНГ ДЛЯ КАЖДОЙ КНИГИ
(вывод первых 5 строк)


,book_id,title,count_review,avg_rating
940,967,Welcome to Temptation (Dempseys #1),2,5.0
300,553,School's Out—Forever (Maximum Ride #2),3,5.0
943,972,Wherever You Go There You Are: Mindfulness Me...,2,5.0
709,418,March,2,5.0
710,421,Marvel 1602,2,5.0


Максимальный рейтинг `5.0` у 44 книг.


Максимальное колчиество обзоров (7) всего у одной книги: "Twilight (Twilight #1)". При этом, книга имеет только средний рейтинг (3.7).

Из 1000 книг датасета 44 книги имеют максимальный рейтинг (5.0).

### Топовое издательство

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [16]:
# создание SQL-запроса
query = '''
SELECT  p.publisher,
        COUNT(*) AS count_book
FROM  books b
LEFT JOIN publishers p ON b.publisher_id=p.publisher_id
WHERE b.num_pages>=50
GROUP BY p.publisher_id, p.publisher
ORDER BY count_book DESC
LIMIT 1

'''

# получение результата запроса
count_book = def_get_sql(query)['count_book'].values[0]
publisher = def_get_sql(query)['publisher'].values[0]

# вывод результата запроса
print(f"Больше всего книг толще 50 страниц выпустило издательство `{publisher}` - {count_book} книги.")

Больше всего книг толще 50 страниц выпустило издательство `Penguin Books` - 42 книги.


Penguin Books — британское издательство, основанное в 1935 году.

### Топовый автор

Определим автора с самой высокой средней оценкой книг для книг с 50 и более оценками.

In [17]:
# создание SQL-запроса
query = '''
SELECT  a.author,
        AVG(rt.rating) AS avg_rating
FROM  books b
LEFT JOIN authors a ON b.author_id=a.author_id
LEFT JOIN ratings rt ON b.book_id=rt.book_id
WHERE b.book_id in (
    SELECT book_id
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating_id)>=50
                   )
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 1

'''

# получение результата запроса
avg_rating = round(def_get_sql(query)['avg_rating'].values[0], 1)
author = def_get_sql(query)['author'].values[0]

# вывод результата запроса
print(f"Самая высокая средняя оценка ({avg_rating}) у `{author}`.")

Самая высокая средняя оценка (4.3) у `J.K. Rowling/Mary GrandPré`.


Наибольшую среднюю оценку получила пара писатель-иллюстратор J.K. Rowling/Mary GrandPré.  
Роулинг - автор серии книг про Гарри Поттера, а Мэри Гран-Пре иллюстрировала эти книги.

### Среднее количество обзоров от пользователей

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [18]:
# создание SQL-запроса
query = '''
SELECT COUNT(DISTINCT re.text)/ COUNT(DISTINCT re.username) AS avg_rev
FROM reviews AS re
JOIN ratings AS r ON r.username = re.username
WHERE re.username IN (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT (rating) > 48)

'''

# получение результата запроса
result = def_get_sql(query)['avg_rev'].values[0]

# вывод результата запроса
print(f"В среднем, пользователи поставившие более 48 оценок книгам и брошюрам, написали {result} обзора.")

В среднем, пользователи поставившие более 48 оценок книгам и брошюрам, написали 24 обзора.


<div style="border:solid green 2px; padding: 20px">

<b>ВЫВОДЫ по разделу</b><br><br>
       
С помощью SQL-запросов к удалённой базе данных была получена следующая информация:

(1) Определено количество книг, вышедших после 1 января 2020 года (821 книга).

(2) Для каждой книги посчитаны количество обзоров - максимальное количество обзоров (7) у одной книги. Также для каждой книги посчитан средний рейтинг (44 книги с максимальным рейтингом "5.0")
    
(3) Определено топовое издательство, выпустившее максимальнео количество книг, толщиной более 50 страниц: "Penguin Books".
    
(4) Определён топовый автор (для книг с 50 и более оценками): Джоан Роулинг (автор серии книг про Гарри Поттера).
    
(5) Для пользователей, которые поставили более 48 оценок, посчитано среднее количество написанных ими обзоров: 24 обзора.
    
</div>

## Выводы

<div style="border:solid green 2px; padding: 20px">

<b>ВЫВОДЫ</b><br><br>
    
    
В рамках данного исследования было выполнено подключение к удалённой базе данных с помощью SQL-запросов.

    Сделан обзор отдельных таблиц базы данных:

- `books`(содержит 1000 книг, 636 авторов и 340 издательств)
- `authors` (содержит 636 авторов)
- `publishers` (содержит 340 издательств)
- `ratings` (содержит информацию о 6456 оценках для 1000 книг от 160 пользователей)
- `review` (содержит 2793 обзора на 994 книги от 160 пользователей)
Для каждой таблицы были выведены первые 5 строк и общая количественная информация.
    
    
Далее с помощью SQL-запросов к удалённой базе данных была получена следующая информация:

(1) Определено количество книг, вышедших после 1 января 2020 года (821 книга).

(2) Для каждой книги посчитаны количество обзоров - максимальное количество обзоров (7) у одной книги. Также для каждой книги посчитан средний рейтинг (44 книги с максимальным рейтингом "5.0")
    
(3) Определено топовое издательство, выпустившее максимальнео количество книг, толщиной более 50 страниц: "Penguin Books".
    
(4) Определён топовый автор (для книг с 50 и более оценками): Джоан Роулинг (автор серии книг про Гарри Поттера).
    
(5) Для пользователей, которые поставили более 48 оценок, посчитано среднее количество написанных ими обзоров: 24 обзора.
    
</div>